## 10-fold Cross Validation on the Stanik dataset

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 29.8 MB/s 
     |████████████████████████████████| 1.9 MB 82.3 MB/s 
     |████████████████████████████████| 1.3 MB 80.0 MB/s 
     |████████████████████████████████| 5.5 MB 82.0 MB/s 
     |████████████████████████████████| 7.6 MB 66.6 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 10.3 MB 75.3 MB/s 
     |████████████████████████████████| 451 kB 91.5 MB/s 
     |████████████████████████████████| 182 kB 90.6 MB/s 
     |████████████████████████████████| 168 kB 92.0 MB/s 
     |████████████████████████████████| 182 kB 85.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 168 kB 85.4 MB/s 
     |████████████████████████████████| 166 kB 88.6 MB/s 
     |████████████████████████████████| 166 kB 87.7 MB/s 
     |████████████████████████████

In [ ]:
# Import downsampled dataset
df = pd.read_excel('DATASET_downsampled_test5.xlsx')

df = df.drop(columns = ['Unnamed: 0'])

df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       2592 non-null   object
 1   kano_labels  2592 non-null   object
 2   labels       2592 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 60.9+ KB


In [ ]:
# Show the distribution of the labels
print(df['labels'].value_counts())

2    648
1    648
3    648
0    648
Name: labels, dtype: int64


In [ ]:
# Rename the columns so that simple transformers can handle them
train_data = df[['review','labels']]
train_data.columns = ['text', 'labels']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2592 non-null   object
 1   labels  2592 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 40.6+ KB


In [ ]:
# Model implementation and k-fold cross validation with Simpletransformers

# Wrapper function to calculate f1-score with additional parameters
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average = None)

# Wrapper function to calculate recall with additional parameters
def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average = None)

# Wrapper function to calculate precision with additional parameters
def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average = None)

# K-fold cross validation with 10 folds
n = 10
kf = KFold(n_splits=n, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc = []
resultsf1 = []
resultsPrecision = []
resultsRecall = []

model_args = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create the ClassificationModel
model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf.split(train_data):
  train_df = train_data.iloc[train_index]
  val_df = train_data.iloc[val_index]

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)
  print(result)
  
  # Add scores to the respective lists
  resultsAcc.append(result['acc'])
  resultsf1.append(result['f1'])
  resultsPrecision.append(result['precision'])
  resultsRecall.append(result['recall'])

  print("results accuracy",resultsAcc)
  print(f"Mean-Precision: {sum(resultsAcc) / len(resultsAcc)}")
  print('results f1: ', resultsf1)
  print('results precision: ', resultsPrecision)
  print('results recall: ', resultsRecall)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

{'mcc': 0.9795426013975215, 'f1': array([0.97520661, 0.99328859, 0.99236641, 0.97435897]), 'acc': 0.9845559845559846, 'recall': array([0.9516129 , 1.        , 0.98484848, 1.        ]), 'precision': array([1.        , 0.98666667, 1.        , 0.95      ]), 'eval_loss': 0.045854324794190965}
results accuracy [0.7538461538461538, 0.8346153846153846, 0.9073359073359073, 0.9073359073359073, 0.918918918918919, 0.9459459459459459, 0.9768339768339769, 0.9806949806949807, 0.9845559845559846]
Mean-Precision: 0.9122314622314625
results f1:  [array([0.89208633, 0.71014493, 0.76595745, 0.65771812]), array([0.93846154, 0.79432624, 0.86614173, 0.73770492]), array([0.9516129 , 0.875     , 0.94656489, 0.85925926]), array([0.98550725, 0.84297521, 0.94366197, 0.83760684]), array([0.97709924, 0.90066225, 0.95384615, 0.83018868]), array([0.96551724, 0.94964029, 0.95934959, 0.91428571]), array([0.97810219, 0.98529412, 0.96721311, 0.97560976]), array([0.99236641, 0.96296296, 0.99280576, 0.97142857]), array([0

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/292 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

{'mcc': 0.9742471473700218, 'f1': array([0.96969697, 0.98611111, 0.99099099, 0.97709924]), 'acc': 0.9806949806949807, 'recall': array([0.96969697, 1.        , 0.98214286, 0.96969697]), 'precision': array([0.96969697, 0.97260274, 1.        , 0.98461538]), 'eval_loss': 0.1377400632455564}
results accuracy [0.7538461538461538, 0.8346153846153846, 0.9073359073359073, 0.9073359073359073, 0.918918918918919, 0.9459459459459459, 0.9768339768339769, 0.9806949806949807, 0.9845559845559846, 0.9806949806949807]
Mean-Precision: 0.9190778140778143
results f1:  [array([0.89208633, 0.71014493, 0.76595745, 0.65771812]), array([0.93846154, 0.79432624, 0.86614173, 0.73770492]), array([0.9516129 , 0.875     , 0.94656489, 0.85925926]), array([0.98550725, 0.84297521, 0.94366197, 0.83760684]), array([0.97709924, 0.90066225, 0.95384615, 0.83018868]), array([0.96551724, 0.94964029, 0.95934959, 0.91428571]), array([0.97810219, 0.98529412, 0.96721311, 0.97560976]), array([0.99236641, 0.96296296, 0.99280576, 0.97

In [ ]:
print("results accuracy",resultsAcc)
print(f"Mean-accuracy: {sum(resultsAcc) / len(resultsAcc)}")
print('results f1: ', resultsf1)
print('results precision: ', resultsPrecision)
print('results recall: ', resultsRecall)

results accuracy [0.7538461538461538, 0.8346153846153846, 0.9073359073359073, 0.9073359073359073, 0.918918918918919, 0.9459459459459459, 0.9768339768339769, 0.9806949806949807, 0.9845559845559846, 0.9806949806949807]
Mean-accuracy: 0.9190778140778143
results f1:  [array([0.89208633, 0.71014493, 0.76595745, 0.65771812]), array([0.93846154, 0.79432624, 0.86614173, 0.73770492]), array([0.9516129 , 0.875     , 0.94656489, 0.85925926]), array([0.98550725, 0.84297521, 0.94366197, 0.83760684]), array([0.97709924, 0.90066225, 0.95384615, 0.83018868]), array([0.96551724, 0.94964029, 0.95934959, 0.91428571]), array([0.97810219, 0.98529412, 0.96721311, 0.97560976]), array([0.99236641, 0.96296296, 0.99280576, 0.97142857]), array([0.97520661, 0.99328859, 0.99236641, 0.97435897]), array([0.96969697, 0.98611111, 0.99099099, 0.97709924])]
results precision:  [array([0.89855072, 0.63636364, 0.8372093 , 0.69014085]), array([0.96825397, 0.69135802, 0.96491228, 0.76271186]), array([0.93650794, 0.8358209 ,

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

def PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1):

  np_resultsPrecision = np.array(resultsPrecision)
  np_resultsRecall = np.array(resultsRecall)
  np_resultsf1  = np.array(resultsf1)

  # Calculate the mean precision for each label
  df_resultsPrecision = pd.DataFrame(np_resultsPrecision, columns=list('BDIP'))
  precision_b_mean = df_resultsPrecision['B'].mean()
  precision_d_mean = df_resultsPrecision['D'].mean()
  precision_i_mean = df_resultsPrecision['I'].mean()
  precision_p_mean = df_resultsPrecision['P'].mean()

  # Calculate the mean recall for each label
  df_resultsRecall = pd.DataFrame(np_resultsRecall, columns=list('BDIP'))
  recall_b_mean = df_resultsRecall['B'].mean()
  recall_d_mean = df_resultsRecall['D'].mean()
  recall_i_mean = df_resultsRecall['I'].mean()
  recall_p_mean = df_resultsRecall['P'].mean()

  # Calculate the mean f1-score for each label
  df_resultsf1 = pd.DataFrame(np_resultsf1, columns=list('BDIP'))
  f1_b_mean = df_resultsf1['B'].mean()
  f1_d_mean = df_resultsf1['D'].mean()
  f1_i_mean = df_resultsf1['I'].mean()
  f1_p_mean = df_resultsf1['P'].mean()

  # Create dataframe with summarized results
  metrics = [{'B': precision_b_mean, 'D': precision_d_mean, 'I': precision_i_mean, 'P':precision_p_mean},
          {'B': recall_b_mean, 'D': recall_d_mean, 'I': recall_i_mean, 'P': recall_p_mean},
          {'B': f1_b_mean, 'D': f1_d_mean, 'I': f1_i_mean, 'P': f1_p_mean} ]
  train_df_metrics = pd.DataFrame(metrics, index=['Average precision',
                                'Average recall',
                                'Average F1-Score'])
  return train_df_metrics

PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1)

,B,D,I,P
Average precision,0.960717,0.868104,0.974059,0.883230
Average recall,0.965020,0.939010,0.906093,0.864989
Average F1-Score,0.962566,0.900041,0.937890,0.873526


## Testing on the Brunotte dataset

In [ ]:
# Import of 'Trainingskorpus_Final', which gives the final labeling
testset1 = pd.read_excel('Trainingskorpus_Final.xlsx')

testset1 = testset1.drop(columns = ['Unnamed: 0'])
testset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
dtypes: object(2)
memory usage: 25.5+ KB


In [ ]:
# Show the distribution of the labels
print(testset1['kano_labels'].value_counts())

B    1102
P     395
D      95
I      30
Name: kano_labels, dtype: int64


In [ ]:
# Import of 'Trainingskorpus_InitialLabels', which gives the initial labeling of two sometimes differing labels
testset2 = pd.read_excel('Trainingskorpus_InitialLabels.xlsx')

testset2 = testset2.drop(columns = ['Unnamed: 0'])
testset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   1622 non-null   object
 1   label 1  1622 non-null   object
 2   label 2  1622 non-null   object
dtypes: object(3)
memory usage: 38.1+ KB


In [ ]:
# Merge the two sets so we have the final labels as well as the initial labels in the testset
testset = testset1.merge(testset2, left_on='review', right_on='review')
testset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1622 entries, 0 to 1621
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
 2   label 1      1622 non-null   object
 3   label 2      1622 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB


In [ ]:
# Add a column 'equal' which is True if the two initial labels are equal and false otherwise
equal = [None]*np.size(testset['review'])

# Add a column 'equal01' which stores binary values (1 if the two initial labels are equal, 0 otherwise) to calculate the phi-coefficient
equal01 = [None]*np.size(testset['review'])

# Add a column 'labeled_correctly01' which stores binary values (1 if the model correctly labeled the text, 0 otherwise) to calculate the phi-coefficient
labeled_correctly01 = [None]*np.size(testset['review'])

# Add a column 'prediction' which contains the model's predictions
prediction = [None]*np.size(testset['review'])

i = 0
for label1, label2 in zip(testset['label 1'], testset['label 2']):
  if(label1 == label2):
    equal[i] = True
    equal01[i] = 1
  else:
    equal[i] = False
    equal01[i] = 0
  i += 1
  
testset['equal'] = equal

In [ ]:
# Calculate the accuracy of the whole testset as well as 
# only the parts where the inital two labels are equal (accuracy_clear)
# or differ (accuracy_diff)

accuracy_clear = 0.
accuracy_diff = 0.
n_clear = 0
n_diff = 0
all_predicted_labels = []
predicted_labels_equal_initial_label = []
predicted_labels_diff_initial_label = []
true_label_equal = []
true_label_diff = []

# counter to go through all the reviews and fill the column 'labeled_correctly' with true if they have been labeled correctly, and false if not
i = 0

for label, review, equal in zip(testset['kano_labels'],  testset['review'], testset['equal']):
  
  test_prediction, raw_outputs = model.predict([review])
  
  predicted_label = ''

  if test_prediction[0] == 0:
    predicted_label = 'B'
    prediction[i] = 'B'
  if test_prediction[0] == 1:
    predicted_label = 'D'
    prediction[i] = 'D'
  if test_prediction[0] == 2:
    predicted_label = 'I'
    prediction[i] = 'I'
  if test_prediction[0] == 3:
    predicted_label = 'P'
    prediction[i] = 'P'

  all_predicted_labels.append(predicted_label)
  
  if equal:
    n_clear += 1
    predicted_labels_equal_initial_label.append(predicted_label)
    true_label_equal.append(label)
    if label == predicted_label:
      accuracy_clear += 1.

  if not equal:
    n_diff += 1
    predicted_labels_diff_initial_label.append(predicted_label)
    true_label_diff.append(label)
    if label == predicted_label:
      accuracy_diff += 1.

  if label == predicted_label:
    labeled_correctly01[i] = 1
  else:
    labeled_correctly01[i] = 0
  i+=1

testset['labeled_correctly'] = labeled_correctly01
testset['prediction'] = prediction

accuracy = (accuracy_clear + accuracy_diff) / (n_clear + n_diff)

accuracy_clear /= n_clear
accuracy_diff /= n_diff

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
testset.to_excel("Brunotte_with_predictions_test5_roberta.xlsx")  

In [ ]:
print('Accuracy Trainingskorpus:                                                  ', round(accuracy*100, 2), '%,  n =', (n_clear + n_diff))
print('Accuracy Trainingskorpus only on the data that coincides on initial labels:', round(accuracy_clear*100, 2), '%,  n =', n_clear)
print('Accuracy Trainingskorpus only on the data that differs on initial labels:  ', round(accuracy_diff*100, 2), '%,  n = ', n_diff)

Accuracy Trainingskorpus:                                                   75.15 %,  n = 1622
Accuracy Trainingskorpus only on the data that coincides on initial labels: 80.4 %,  n = 1383
Accuracy Trainingskorpus only on the data that differs on initial labels:   44.77 %,  n =  239


In [ ]:
# Calculate precision, recall and f1-score for each label

test_df_metrics = precision_recall_fscore_support(testset['kano_labels'], all_predicted_labels, average=None,labels=['B', 'D', 'I', 'P'])

print('Precision B: ', test_df_metrics[0][0])
print('Precision D: ', test_df_metrics[0][1])
print('Precision I: ', test_df_metrics[0][2])
print('Precision P: ', test_df_metrics[0][3])

print('Recall B: ', test_df_metrics[1][0])
print('Recall D: ', test_df_metrics[1][1])
print('Recall I: ', test_df_metrics[1][2])
print('Recall P: ', test_df_metrics[1][3])

print('F1-score B: ', test_df_metrics[2][0])
print('F1-score D: ', test_df_metrics[2][1])
print('F1-score I: ', test_df_metrics[2][2])
print('F1-score P: ', test_df_metrics[2][3])

Precision B:  0.8958333333333334
Precision D:  0.4294117647058823
Precision I:  0.4375
Precision P:  0.5514018691588785
Recall B:  0.8194192377495463
Recall D:  0.7684210526315789
Recall I:  0.23333333333333334
Recall P:  0.5974683544303797
F1-score B:  0.8559241706161138
F1-score D:  0.5509433962264151
F1-score I:  0.3043478260869565
F1-score P:  0.5735115431348724


In [ ]:
# Calculates the phi-coefficient of the labels being predicted correctly and the researchers being 

print('MCC for initial agreement of raters and correct or false labeling',matthews_corrcoef(labeled_correctly01, equal01))

MCC for initial agreement of raters and correct or false labeling 0.2923027788877421


## 10-fold Cross Validation on a combination of the Stanik dataset and the Brunotte dataset

In [ ]:
# Import combined downsampled dataset 
df_combined = pd.read_excel('DATASET_Trainingskorpus_combined_test5.xlsx')

df_combined = df_combined.drop(columns = ['Unnamed: 0'])

# Rename column so that simple transformers can handle it
df_combined = df_combined.rename(columns = {'review': 'text'})
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         2972 non-null   object
 1   kano_labels  2972 non-null   object
 2   labels       2972 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 69.8+ KB


In [ ]:
# Show the distribution of the labels
print(df_combined['labels'].value_counts())

0    743
1    743
3    743
2    743
Name: labels, dtype: int64


In [ ]:
# 10-fold cross validation on the combined dataset

n2 = 10
kf2 = KFold(n_splits=n2, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc2 = []
resultsf12 = []
resultsPrecision2 = []
resultsRecall2 = []

model_args2 = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create a ClassificationModel
model2 = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf2.split(df_combined):
  train_df2 = df_combined.iloc[train_index]
  val_df2 = df_combined.iloc[val_index]

  model2.train_model(df_combined)
  result2, model_outputs2, wrong_predictions2 = model2.eval_model(val_df2, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)

  print(result2)
  # Add scores to the respective lists
  resultsAcc2.append(result2['acc'])
  resultsf12.append(result2['f1'])
  resultsPrecision2.append(result2['precision'])
  resultsRecall2.append(result2['recall'])

  print("results accuracy",resultsAcc2)
  print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
  print('results f1: ', resultsf12)
  print('results precision: ', resultsPrecision2)
  print('results recall: ', resultsRecall2)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.751266644305803, 'f1': array([0.86153846, 0.78980892, 0.875     , 0.73939394]), 'acc': 0.8120805369127517, 'recall': array([0.91803279, 0.84931507, 0.80769231, 0.70930233]), 'precision': array([0.8115942 , 0.73809524, 0.95454545, 0.7721519 ]), 'eval_loss': 0.50257366581967}
results accuracy [0.8120805369127517]
Mean-accuracy: 0.8120805369127517
results f1:  [array([0.86153846, 0.78980892, 0.875     , 0.73939394])]
results precision:  [array([0.8115942 , 0.73809524, 0.95454545, 0.7721519 ])]
results recall:  [array([0.91803279, 0.84931507, 0.80769231, 0.70930233])]


...

{'mcc': 0.9774004771045751, 'f1': array([0.99337748, 0.97382199, 0.99224806, 0.97560976]), 'acc': 0.9831649831649831, 'recall': array([0.98684211, 0.96875   , 0.98461538, 1.        ]), 'precision': array([1.        , 0.97894737, 1.        , 0.95238095]), 'eval_loss': 0.08066084431974511}
results accuracy [0.8120805369127517, 0.8691275167785235, 0.898989898989899, 0.9326599326599326, 0.9696969696969697, 0.9797979797979798, 0.9764309764309764, 0.9831649831649831, 0.9831649831649831]
Mean-accuracy: 0.933901530844111
results f1:  [array([0.86153846, 0.78980892, 0.875     , 0.73939394]), array([0.93902439, 0.83950617, 0.91780822, 0.75806452]), array([0.94736842, 0.86567164, 0.95104895, 0.82191781]), array([0.99186992, 0.87179487, 0.96202532, 0.91719745]), array([0.99236641, 0.95597484, 0.98701299, 0.94666667]), array([0.98550725, 0.97142857, 0.99363057, 0.96855346]), array([0.97560976, 0.97058824, 0.99310345, 0.96644295]), array([0.99386503, 0.97810219, 0.99363057, 0.96350365]), array([0.99

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.9820160809601693, 'f1': array([0.98136646, 0.98780488, 0.99186992, 0.98630137]), 'acc': 0.9865319865319865, 'recall': array([0.97530864, 1.        , 0.98387097, 0.98630137]), 'precision': array([0.9875    , 0.97590361, 1.        , 0.98630137]), 'eval_loss': 0.09096611448024448}
results accuracy [0.8120805369127517, 0.8691275167785235, 0.898989898989899, 0.9326599326599326, 0.9696969696969697, 0.9797979797979798, 0.9764309764309764, 0.9831649831649831, 0.9831649831649831, 0.9865319865319865]
Mean-accuracy: 0.9391645764128984
results f1:  [array([0.86153846, 0.78980892, 0.875     , 0.73939394]), array([0.93902439, 0.83950617, 0.91780822, 0.75806452]), array([0.94736842, 0.86567164, 0.95104895, 0.82191781]), array([0.99186992, 0.87179487, 0.96202532, 0.91719745]), array([0.99236641, 0.95597484, 0.98701299, 0.94666667]), array([0.98550725, 0.97142857, 0.99363057, 0.96855346]), array([0.97560976, 0.97058824, 0.99310345, 0.96644295]), array([0.99386503, 0.97810219, 0.99363057, 0.96

In [ ]:
print("results accuracy",resultsAcc2)
print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
print('results f1: ', resultsf12)
print('results precision: ', resultsPrecision2)
print('results recall: ', resultsRecall2)

results accuracy [0.8120805369127517, 0.8691275167785235, 0.898989898989899, 0.9326599326599326, 0.9696969696969697, 0.9797979797979798, 0.9764309764309764, 0.9831649831649831, 0.9831649831649831, 0.9865319865319865]
Mean-accuracy: 0.9190778140778143
results f1:  [array([0.86153846, 0.78980892, 0.875     , 0.73939394]), array([0.93902439, 0.83950617, 0.91780822, 0.75806452]), array([0.94736842, 0.86567164, 0.95104895, 0.82191781]), array([0.99186992, 0.87179487, 0.96202532, 0.91719745]), array([0.99236641, 0.95597484, 0.98701299, 0.94666667]), array([0.98550725, 0.97142857, 0.99363057, 0.96855346]), array([0.97560976, 0.97058824, 0.99310345, 0.96644295]), array([0.99386503, 0.97810219, 0.99363057, 0.96350365]), array([0.99337748, 0.97382199, 0.99224806, 0.97560976]), array([0.98136646, 0.98780488, 0.99186992, 0.98630137])]
results precision:  [array([0.8115942 , 0.73809524, 0.95454545, 0.7721519 ]), array([0.89534884, 0.75555556, 0.97101449, 0.88679245]), array([0.94186047, 0.82857143,

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

PrecisionRecallF1Table(resultsPrecision2, resultsRecall2, resultsf12)

,B,D,I,P
Average precision,0.960947,0.894642,0.985874,0.922914
Average recall,0.972619,0.950221,0.947785,0.889804
Average F1-Score,0.966189,0.920450,0.965738,0.904365
